<a href="https://colab.research.google.com/github/KarthikeyanBaskaran/pdf-build/blob/main/ReportLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install reportlab pyyaml

In [12]:
# COrrect code

import yaml
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, ListFlowable, ListItem, HRFlowable
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch


# Load YAML
def load_content(yaml_file):
    with open(yaml_file, "r") as f:
        return yaml.safe_load(f)

def build_pdf(data, output_file="resume.pdf"):
    doc = SimpleDocTemplate(output_file, pagesize=A4,
                            leftMargin=40, rightMargin=40,
                            topMargin=40, bottomMargin=40)
    elements = []
    styles = getSampleStyleSheet()

    # Custom styles
    section_style = ParagraphStyle("Section", fontSize=14, textColor=colors.black, spaceAfter=6, leading=16, spaceBefore=12)
    text_style = ParagraphStyle("Text", fontSize=10, leading=14)
    bullet_style = ParagraphStyle("Bullet", fontSize=10, leading=14, leftIndent=15, bulletIndent=5)

    # ==== HEADER (from Code B) ====
    big_bold_style = ParagraphStyle(
        name='BigBold',
        parent=styles['Normal'],
        fontName='Helvetica-Bold',
        fontSize=18,
        leading=22,
        textColor=colors.black,
        alignment=0  # Left aligned
    )

    left_normal_style = ParagraphStyle(
        name='LeftNormal',
        parent=styles['Normal'],
        fontName='Helvetica',
        fontSize=10,
        leading=12,
        textColor=colors.black,
        alignment=0
    )

    right_normal_style = ParagraphStyle(
        name='RightNormal',
        parent=styles['Normal'],
        fontName='Helvetica',
        fontSize=10,
        leading=12,
        textColor=colors.black,
        alignment=2  # Right aligned
    )

    # Left and right column data
    left_col = [
        Paragraph('Karthik', big_bold_style),
        Paragraph('(Karthikeyan Baskaran)', left_normal_style),
        Paragraph('', left_normal_style),
        Paragraph('', left_normal_style)
    ]

    right_col = [
        Paragraph('+1 437-665-6654', right_normal_style),
        Paragraph('karthikeyanbaskarca@gmail.com', right_normal_style),
        Paragraph('https://github.com/KarthikeyanBaskaran', right_normal_style),
        Paragraph('https://www.linkedin.com/in/karthikeyan-baskaran-data/', right_normal_style)
    ]

    # Combine into table
    header_data = [[left, right] for left, right in zip(left_col, right_col)]
    header_table = Table(header_data, colWidths=[2.5*inch, 4*inch], hAlign='CENTER')

    header_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.transparent),
        ('LEFTPADDING', (0, 0), (-1, -1), 0),
        ('RIGHTPADDING', (0, 0), (-1, -1), 0),
        ('TOPPADDING', (0, 0), (-1, -1), 0),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 0),
    ]))

    # elements.append(Spacer(1, 0.3*inch))
    elements.append(header_table)
    # elements.append(Spacer(1, 12))

    # ---- SUMMARY ----
    elements.append(Paragraph("Summary", section_style))

    elements.append(HRFlowable(width="100%", thickness=0.75, lineCap='round', color='black'))
    elements.append(Spacer(1, 6))

    elements.append(Paragraph(data["summary"], text_style))

    # ---- SKILLS ----
    elements.append(Paragraph("Skills", section_style))

    elements.append(HRFlowable(width="100%", thickness=0.75, lineCap='round', color='black'))
    elements.append(Spacer(1, 6))

    for item in data.get("skills", []):
        category = item.get("name", "Skills")
        keywords = item.get("keywords", [])
        elements.append(Paragraph(f"<b>{category}:</b> {', '.join(map(str, keywords))}", text_style))

    # ---- WORK EXPERIENCE ----
    elements.append(Paragraph("Work Experience", section_style))

    elements.append(HRFlowable(width="100%", thickness=0.75, lineCap='round', color='black'))
    elements.append(Spacer(1, 6))

    hardcoded_jobs = [
        {"title": "Data Analyst", "company": "Vestas Wind Systems", "dates": "Jan 2020 - Dec 2022"},
        {"title": "Project Coordinator", "company": "ManpowerGroup Services", "dates": "Jan 2018 - Dec 2019"},
        {"title": "Manufacturing Analyst", "company": "Valeo India", "dates": "Jan 2015 - Dec 2017"}
    ]

    for idx, job in enumerate(hardcoded_jobs):
        elements.append(Paragraph(f"<b>{job['title']}</b> | {job['company']}", text_style))
        elements.append(Paragraph(job["dates"], text_style))

        yaml_job = data['work_experience'][idx]
        bullet_points = [ListItem(Paragraph(bp, bullet_style)) for bp in yaml_job["achievements"]]

        elements.append(ListFlowable(bullet_points, bulletType="bullet", start="•"))
        elements.append(Spacer(1, 6))

    # ---- PROJECTS ----
    elements.append(Paragraph("Projects", section_style))

    elements.append(HRFlowable(width="100%", thickness=0.75, lineCap='round', color='black'))
    elements.append(Spacer(1, 6))

    for proj in data["projects"]:
        elements.append(Paragraph(f"<b>{proj['project_name']}</b>", text_style))
        elements.append(Paragraph(proj["description"], text_style))
        elements.append(Paragraph(f"({', '.join(proj['keywords'])})", text_style))
        elements.append(Spacer(1, 6))

    # ---- EDUCATION ----
    elements.append(Paragraph("Education", section_style))

    elements.append(HRFlowable(width="100%", thickness=0.75, lineCap='round', color='black'))
    elements.append(Spacer(1, 6))

    hardcoded_education = [
        {"degree": "Diploma in Data Analytics for Business", "institution": "St. Clair College, Canada", "gpa": "4.0 GPA", "dates": "Sep 2023 - Apr 2025"},
        {"degree": "Bachelor of Mechanical Engineering", "institution": "St. Joseph's Institute of Technology, India", "gpa": "3.7 GPA", "dates": "Jun 2014 - Jun 2018"}
    ]

    for edu in hardcoded_education:
        elements.append(Paragraph(f"<b>{edu['degree']}</b> - {edu['institution']}", text_style))
        elements.append(Paragraph(f"GPA: {edu['gpa']} | {edu['dates']}", text_style))
        elements.append(Spacer(1, 6))

    # Build PDF
    doc.build(elements)
    print(f"✅ Resume generated: {output_file}")

if __name__ == "__main__":
    content = load_content("output.yaml")
    build_pdf(content, "formatted_resume.pdf")


✅ Resume generated: formatted_resume.pdf


In [2]:
from reportlab.pdfgen import canvas

c = canvas.Canvas("line_example.pdf")

# Draw a horizontal line (x1, y1, x2, y2)
c.line(50, 750, 550, 750)

c.save()
